# Backtest

In [ ]:
from system.agent import Agent
from system.env import Env, GEnv

from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


START = "2022-01-01"
END = "2023-10-01"
TICKERS = ["BTC", "ETH", "SOL", "DASH", "MATIC", "TWT", "GALA", "EGLD", "XMR"]

INTERVAL = "1d"
CAPITAL = 100

config = {
    "symbols" : TICKERS, "interval" : INTERVAL, "start" : START, "end" : END
    }
env = GEnv(**config)


sub_env_1 = Env(symbol="BTC", interval=INTERVAL, start=START, end=END)
sub_env_2 = Env(symbol="ETH", interval=INTERVAL, start=START, end=END)
sub_env_3 = Env(symbol="SOL", interval=INTERVAL, start=START, end=END)
sub_env_4 = Env(symbol="EGLD", interval=INTERVAL, start=START, end=END)


# Agent details
CAPITAL = 100
AGENT_PARAMS = [
    [(1, "BTC"), CAPITAL, sub_env_1],
    [(2, "ETH"), CAPITAL,sub_env_2],
    [(3, "SOL"), CAPITAL,sub_env_3],
    [(4, "EGLD"), CAPITAL,sub_env_4]
]


STRATEGIES = [
    ("MOMENTUM", 3, {"floor" : 4}),
    ("TRIPLEMA", (3, 7, 14), {"floor" : 4}),
    ("MOMENTUM", 8, {"floor" : 4}),
    ("TRIPLEMA", (14, 21, 35), {"floor" : 4})
]

AGENT = []
for agent_x, strategie in zip(AGENT_PARAMS, STRATEGIES):
    clear_output()
    print(agent_x)
    agent = Agent(*agent_x)
    agent.update_policy(*strategie)
    agent.run_episode()
    AGENT.append(agent)
    
    

In [ ]:
AGENT[0].view_report()

In [ ]:
#AGENT[1].env.post_event.tradeData[AGENT[1].env.post_event.tradeData["session"]==1]
AGENT[1].env.post_event.tradeData.iloc[:53]

In [ ]:
AGENT[1].view_report()

In [ ]:
AGENT[3].view_report()

In [ ]:
#AGENT[3].post_event.tradeData[AGENT[3].post_event.tradeData["session"] == 1].iloc[2:12]

In [ ]:
env.globalReport()

# Agent : Learning

In [27]:
from system.agent import Agent
from system.env import Env, GEnv

from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


START = "2023"
END = "2024"
TICKERS = ["BTC", "ETH", "SOL", "QNT", "DASH", "MATIC", "TWT", "GALA", "EGLD", "XMR"]

SYMBOL = "QNT"
INTERVAL = "1d"
CAPITAL = 1000


config = {
    "symbols" : TICKERS, "interval" : INTERVAL, "start" : START, "end" : END
    }
env = GEnv(**config)

sub_env = Env(symbol=SYMBOL, interval=INTERVAL, start=START, end=END)

# Agent details
CAPITAL = 1000
LEVERAGE = 5

n_SESSION = 100

AGENT_PARAMS = [
    [(1, SYMBOL), CAPITAL, sub_env, n_SESSION]
]




STRATEGIES = [
    ("MOMENTUM", 3, {"floor" : 4}),
    ("TRIPLEMA", (3, 7, 14), {"floor" : 4}),
    ("MOMENTUM", 8, {"floor" : 4}),
    ("TRIPLEMA", (14, 21, 35), {"floor" : 4})
]

STRATEGIES = [
    ("TRIPLEMA", (3, 7, 14), {"floor" : 4})
]

STRATEGIES = [
    ("MOMENTUM", 3, {"floor" : 4})
]

AGENT = []
for agent_x, strategie in zip(AGENT_PARAMS, STRATEGIES):
    print(agent_x)
    agent = Agent(*agent_x)
    agent.update_policy(*strategie)
    agent.run_episode()
    AGENT.append(agent)
    
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[(1, 'QNT'), 1000, <system.env.Env object at 0x000001387A4B5590>, 100]
{'date': Timestamp('2023-04-09 00:00:00'), 'nbTrades': 8, 'winRate': 0.25, 'lossRate': -0.75, 'amountWin': 99.90304555826776, 'amountLoss': -202.32851376158192, 'totalAmount': -102.42546820331415, 'amount': -102.42546820331415, 'expectancy': -12.80318352541427, 'profitFactor': 0.4937665171404888, 'minExposure': 3, 'maxExposure': 15}
__ End Session 0 ____________
__ Start Session 1____________
{'date': Timestamp('2023-07-19 00:00:00'), 'nbTrades': 19, 'winRate': 0.42105263157894735, 'lossRate': -0.5789473684210527, 'amountWin': 287.8060130833961, 'amountLoss': -162.05275694789202, 'totalAmount': 125.7532561355041, 'amount': 23.32778793218995, 'expectancy': 6.618592428184424, 'profitFactor': 1.776001954572979, 'minExposure': 3, 'maxExposure': 13}
__ End Session 1 ____________
__ Start Session 2____________
{'date': Timestamp('2023-

In [28]:
AGENT[0].view_report()

(1, 'QNT')


In [20]:
session = AGENT[0].post_event.sessionData
session

,date,nbTrades,winRate,lossRate,amountWin,amountLoss,totalAmount,amount,expectancy,profitFactor,minExposure,maxExposure
0,2024-03-05 02:00:00,11,0.363636,-0.636364,22.225746,-66.321304,-44.095559,-44.095559,-4.008687,0.335122,3,9
1,2024-03-09 07:00:00,22,0.272727,-0.727273,180.522764,-178.709870,1.812894,-42.282664,0.082404,1.010144,3,10
2,2024-03-13 12:00:00,19,0.421053,-0.578947,116.357927,-166.633838,-50.275911,-92.558576,-2.646101,0.698285,3,11
3,2024-03-17 17:00:00,26,0.115385,-0.884615,46.233477,-341.176452,-294.942975,-377.893731,-11.343961,0.135512,3,12
4,2024-03-21 21:00:00,18,0.444444,-0.555556,165.160508,-116.325955,48.834553,-338.666997,2.713031,1.419808,3,14
5,2024-03-26 03:00:00,20,0.250000,-0.750000,92.161754,-88.548436,3.613319,-333.644894,0.180666,1.040806,3,12
6,2024-03-30 08:00:00,23,0.260870,-0.739130,68.976691,-83.853623,-14.876932,-349.930611,-0.646823,0.822585,3,12
7,2024-04-03 13:00:00,24,0.250000,-0.750000,28.074817,-115.418688,-87.343871,-428.079620,-3.639328,0.243243,3,8
8,2024-04-07 18:00:00,22,0.272727,-0.727273,36.329822,-53.521976,-17.192154,-450.392112,-0.781462,0.678783,3,14
9,2024-04-11 23:00:00,21,0.333333,-0.666667,22.794969,-44.997726,-22.202757,-476.669393,-1.057274,0.506580,3,13


In [3]:
(session["nbTrades"].sum() / dd.shape[0]) * 100

KeyError: 'nbTrades'

In [21]:
dd = AGENT[0].env.post_event.tradeData
dd.head(15)

,agentId,date,price,quantity,position,side,state,in_value,out_value,value,...,pnl_pct,symbol,session,status,gp,cum_gp,ret_price,cum_price,loss,recovery
0,1,2024-03-03 01:00:00,126.9,7.880221,1,LONG,"('Open', 'LONG', False, False)",1000.000000,0.000000,1000.000000,...,0.000000,QNT,0,Open,0.000000,0.000000,NaN,NaN,0.000000,0.000000
1,1,2024-03-03 02:00:00,127.2,0.000000,0,None,"('Close', 'LONG', False, False)",0.000000,1002.364066,0.000000,...,0.002364,QNT,0,Close,2.364066,2.364066,0.002364,1.002364,0.000000,0.000000
2,1,2024-03-03 03:00:00,127.8,0.000000,0,None,"('-', None, False, False)",0.000000,0.000000,0.000000,...,0.000000,QNT,0,-,0.000000,2.364066,0.004717,1.007092,0.000000,0.000000
3,1,2024-03-03 04:00:00,129.4,7.746245,1,LONG,"('Open', 'LONG', False, False)",1002.364066,0.000000,1002.364066,...,0.000000,QNT,0,Open,0.000000,2.364066,0.012520,1.019701,0.000000,0.000000
4,1,2024-03-03 05:00:00,128.1,7.746245,1,LONG,"('-', 'LONG', False, False)",1002.364066,0.000000,992.293948,...,-0.010046,QNT,0,-,-10.070118,-7.706052,-0.010046,1.009456,-0.010046,0.010148
5,1,2024-03-03 06:00:00,128.5,7.746245,1,LONG,"('-', 'LONG', False, False)",1002.364066,0.000000,995.392446,...,-0.006955,QNT,0,-,3.098498,-4.607554,0.003123,1.012608,-0.006955,0.007004
6,1,2024-03-03 07:00:00,125.3,0.000000,0,None,"('Close', 'LONG', False, False)",0.000000,970.604463,0.000000,...,-0.031685,QNT,0,Close,-24.787983,-29.395537,-0.024903,0.987392,-0.031685,0.032721
7,1,2024-03-03 08:00:00,125.9,-7.709329,-1,SHORT,"('Open', 'SHORT', False, False)",970.604463,0.000000,970.604463,...,0.000000,QNT,0,Open,0.000000,-29.395537,0.004789,0.992120,0.000000,0.000000
8,1,2024-03-03 09:00:00,124.8,-7.709329,-1,SHORT,"('-', 'SHORT', False, False)",970.604463,0.000000,979.084724,...,0.008737,QNT,0,-,8.480261,-20.915276,-0.008737,0.983452,0.000000,0.000000
9,1,2024-03-03 10:00:00,124.5,-7.709329,-1,SHORT,"('-', 'SHORT', False, False)",970.604463,0.000000,981.397523,...,0.011120,QNT,0,-,2.312799,-18.602477,-0.002404,0.981087,0.000000,0.000000


In [ ]:
port = AGENT[0].env.post_event.portfolioData
port

#### --------------------------------------

# *Multi-Agent* v2(1)

In [ ]:
from magent import MAgentThread
from master import  MasterAgentThread
from system.env import Env, SubEnv

from threading import Thread, Condition, Event, Barrier, Semaphore

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


TICKERS = ["BTC", "ETH", "SOL", "DASH"]
START = "2022-01-01"
END = "2023-10-01"
INTERVAL = "1d"
CAPITAL = 100

config = {"capital" : CAPITAL, "interval" : INTERVAL,
          "symbols" : TICKERS, "start" : START, "end" : END}
env = Env(**config)

sub_env_1 = SubEnv(symbol="BTC", capital=CAPITAL, interval=INTERVAL, start=START, end=END)
sub_env_2 = SubEnv(symbol="ETH", capital=CAPITAL, interval=INTERVAL, start=START, end=END)
sub_env_3 = SubEnv(symbol="SOL", capital=CAPITAL, interval=INTERVAL, start=START, end=END)
sub_env_4 = SubEnv(symbol="EGLD", capital=CAPITAL, interval=INTERVAL, start=START, end=END)

# Agent details
AGENT_PARAMS = [
    [(1, "BTC"), sub_env_1],
    [(2, "ETH"), sub_env_2],
    [(3, "SOL"), sub_env_3],
    [(4, "EGLD"), sub_env_4]
]

N = len(AGENT_PARAMS)

STRATEGIES = [
    ((1, "BTC"), "MOMENTUM", 3),
    ((2, "ETH"), "TRIPLEMA", (3, 7, 14)),
    ((3, "SOL"), "MOMENTUM", 8),
    ((4, "EGLD"), "TRIPLEMA", (14, 21, 35))
]

In [ ]:
master = MasterAgentThread(env)
master.set_barrier(n=N)

for agent_param, strategie in zip(AGENT_PARAMS, STRATEGIES):
    master.add_agent(*agent_param)
    master.update_params(*strategie)


master.active_agents()
master.run()

In [ ]:
master.agents

In [ ]:
#master.agents[(1, "BTC")].env.get_viz(1, "BTC")
master.agents[(AGENT_PARAMS[0][0])].env.get_viz(*AGENT_PARAMS[0][0])

In [ ]:
master.agents[(AGENT_PARAMS[1][0])].env.get_viz(*AGENT_PARAMS[1][0])

In [ ]:
master.agents[(AGENT_PARAMS[2][0])].env.get_viz(*AGENT_PARAMS[2][0])

In [ ]:
master.agents[(AGENT_PARAMS[3][0])].env.get_viz(*AGENT_PARAMS[3][0])

In [ ]:
master.agents[(4, "EGLD")].env.journal.portfolioData
#master.agents[(3, "SOL")].env.journal.tradesData

In [ ]:
master.agents

In [ ]:
master.agents[(3, "SOL")].env.journal.portfolioData

# *End*

In [ ]:
import threading

threading.enumerate()

# *Multi-Agent v3* Multiprocessing

In [ ]:
import multiprocessing
from multiprocessing import Process, Manager, Queue, Barrier, Event, Semaphore



# Plot

In [ ]:
import plotly.graph_objects as go

categories = ['processing cost','mechanical properties','chemical stability',
              'thermal stability', 'device integration']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=[1, 5, 2, 2, 3],
      theta=categories,
      fill='toself',
      name='Product A'
))
fig.add_trace(go.Scatterpolar(
      r=[4, 3, 2.5, 1, 2],
      theta=categories,
      fill='toself',
      name='Product B'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 5]
    )),
  showlegend=False
)

fig.show()

In [ ]:
class MyGenerator:
    def __init__(self, n):
        self.n = n
        self.current = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.current < self.n:
            result = self.current
            self.current += 1
            return result
        else:
            raise StopIteration

# Créez une instance de la classe MyGenerator
dix_nombres = MyGenerator(10)

# Utilisez une boucle for pour obtenir les valeurs
for nombre in dix_nombres:
    print(nombre)


In [ ]:
import multiprocessing

multiprocessing.cpu_count()

In [ ]:
import os 

os.cpu_count()

In [ ]:
import threading

class Master:
    def __init__(self, condition, event):
        self.condition = condition
        self.event = event

    def run(self):
        for _ in range(10):
            with self.condition:
                print("Maître : J'attends que les agents soient prêts.")
                self.condition.wait()
                print("Maître : Autorisation d'exécuter le travail.")
                self.event.set()
                self.condition.notify_all()

class Agent:
    def __init__(self, agent_id, condition, event):
        self.agent_id = agent_id
        self.condition = condition
        self.event = event

    def run(self):
        for _ in range(10):
            with self.condition:
                print(f"Agent {self.agent_id} : Prêt à travailler.")
                self.condition.notify()
                self.condition.wait()
                self.event.wait()
                self.event.clear()
                print(f"Agent {self.agent_id} : Travail en cours.")
                self.condition.notify()
                self.condition.wait()
                print(f"Agent {self.agent_id} : Travail terminé.")

# Créez une Condition pour coordonner les agents et le maître
condition = threading.Condition()

# Créez un Event pour autoriser les agents à travailler
event = threading.Event()

# Créez le maître et les agents
master = Master(condition, event)
agents = [Agent(i, condition, event) for i in range(1, 4)]

# Créez des threads pour le maître et les agents
master_thread = threading.Thread(target=master.run)
agent_threads = [threading.Thread(target=agent.run) for agent in agents]

# Démarrez les threads
master_thread.start()
for thread in agent_threads:
    thread.start()

# Attendez que tous les threads se terminent
master_thread.join()
for thread in agent_threads:
    thread.join()

print("Tous les agents et le maître ont terminé.")


In [ ]:
import time
from threading import Thread
from queue import Queue

class Agent:
    def __init__(self, master_queue, name):
        self.master_queue = master_queue
        self.name = name

    def do_task(self, task):
        print(f"Agent {self.name} received task: {task}")
        # Simulating task execution
        time.sleep(2)
        print(f"Agent {self.name} completed task: {task}")
        # Inform the master that the task is completed
        self.master_queue.put((self.name, task))


class Master:
    def __init__(self):
        self.agent_queues = {}

    def add_agent(self, agent):
        agent_queue = Queue()
        self.agent_queues[agent.name] = agent_queue
        Thread(target=self.listen_to_agent, args=(agent, agent_queue)).start()

    def assign_task(self, agent_name, task):
        agent_queue = self.agent_queues.get(agent_name)
        if agent_queue:
            print(f"Master assigned task: {task} to Agent {agent_name}")
            agent_queue.put(task)
        else:
            print(f"Agent {agent_name} not found.")

    def listen_to_agent(self, agent, agent_queue):
        while True:
            task = agent_queue.get()
            if task is None:
                break
            print(f"Master received task completion report from Agent {agent.name} for task: {task}")


# Exemple d'utilisation
if __name__ == "__main__":
    master = Master()

    agents = [Agent(master_queue=master, name=f"Agent{i+1}") for i in range(4)]

    for agent in agents:
        master.add_agent(agent)

    master.assign_task("Agent1", "Task1")
    master.assign_task("Agent2", "Task2")
    master.assign_task("Agent3", "Task3")
    master.assign_task("Agent4", "Task4")

    # Attendez la fin des tâches avant de quitter
    time.sleep(3)  # Simuler le temps nécessaire pour terminer les tâches
    for agent in agents:
        master.add_agent(agent)  # Envoyer un signal de fin à tous les agents


In [ ]:
import multiprocessing
import time
import random

class Agent(multiprocessing.Process):
    def __init__(self, agent_id, master_queue, agent_queue):
        super().__init__()
        self.agent_id = agent_id
        self.master_queue = master_queue
        self.agent_queue = agent_queue

    def run(self):
        while True:
            instruction = self.master_queue.get()

            if instruction == "QUIT":
                break

            # Ajoutez des conditions en fonction de l'instruction
            if instruction == "Commande 1":
                result = f"Agent {self.agent_id} exécute la Commande 1 spécifique."
            elif instruction == "Commande 2":
                result = f"Agent {self.agent_id} exécute la Commande 2 spécifique."
            else:
                result = f"Agent {self.agent_id} exécute une instruction non gérée."

            self.agent_queue.put(result)

class Master:
    def __init__(self):
        self.master_queue = multiprocessing.Queue()
        self.agent_queues = [multiprocessing.Queue() for _ in range(4)]
        self.agents = [Agent(i, self.master_queue, self.agent_queues[i]) for i in range(4)]

    def start_agents(self):
        for agent in self.agents:
            agent.start()

    def send_instructions(self, instructions):
        for instruction in instructions:
            for agent_queue in self.agent_queues:
                self.master_queue.put(instruction)

            time.sleep(1)  # Attendez un certain temps pour simuler le traitement parallèle

            for _ in range(4):
                report = self.master_queue.get()
                print(report)

    def stop_agents(self):
        for _ in range(4):
            self.master_queue.put("QUIT")

        for agent in self.agents:
            agent.join()

if __name__ == "__main__":
    master_instance = Master()
    master_instance.start_agents()

    instructions = ["Commande 1", "Commande 2", "Commande 3", "Commande 4"]
    master_instance.send_instructions(instructions)

    master_instance.stop_agents()


In [ ]:
import multiprocessing
import time

class Agent(multiprocessing.Process):
    def __init__(self, agent_id, master_event, agent_queue):
        super().__init__()
        self.agent_id = agent_id
        self.master_event = master_event
        self.agent_queue = agent_queue

    def run(self):
        while True:
            # Wait for the event to be set by the master
            self.master_event.wait()

            instruction = self.agent_queue.get()

            if instruction == "QUIT":
                break

            # Your conditions or logic here
            result = f"Agent {self.agent_id} a exécuté l'instruction: {instruction}"

            # Report the result to the master
            self.agent_queue.put(result)

class Master:
    def __init__(self):
        self.master_event = multiprocessing.Event()
        self.agent_queues = [multiprocessing.Queue() for _ in range(4)]
        self.agents = [Agent(i, self.master_event, self.agent_queues[i]) for i in range(4)]

    def start_agents(self):
        for agent in self.agents:
            agent.start()

    def send_instructions(self, instructions):
        for instruction in instructions:
            # Send the instruction to all agents
            for agent_queue in self.agent_queues:
                agent_queue.put(instruction)

            # Set the event to signal agents to start processing
            self.master_event.set()

            time.sleep(1)  # Simulate parallel processing time

            # Clear the event for the next round
            self.master_event.clear()

            # Collect reports from agents
            for _ in range(4):
                report = self.agent_queues[0].get()  # Assuming all agents provide the same result
                print(report)

    def stop_agents(self):
        # Send the QUIT instruction to all agents
        for agent_queue in self.agent_queues:
            agent_queue.put("QUIT")

        # Wait for agents to finish
        for agent in self.agents:
            agent.join()

if __name__ == "__main__":
    master_instance = Master()
    master_instance.start_agents()

    instructions = ["Commande 1", "Commande 2", "Commande 3", "Commande 4"]
    master_instance.send_instructions(instructions)

    master_instance.stop_agents()


In [ ]:


class ProfitManagement:
    def __init__(self, profit_target_percentage, trailing_stop_percentage):
        self.profit_target_percentage = profit_target_percentage
        self.trailing_stop_percentage = trailing_stop_percentage
        self.entry_price = None
        self.profit_target_price = None
        self.trailing_stop_price = None

    def set_entry_price(self, entry_price):
        self.entry_price = entry_price
        self.profit_target_price = entry_price * (1 + self.profit_target_percentage)
        self.trailing_stop_price = entry_price * (1 - self.trailing_stop_percentage)

    def take_profit(self, current_price):
        if current_price >= self.profit_target_price:
            return True
        return False

    def trailing_stop_loss(self, current_price):
        if current_price >= self.trailing_stop_price:
            self.trailing_stop_price = current_price * (1 - self.trailing_stop_percentage)
        return self.trailing_stop_price

# Exemple d'utilisation
profit_target_percentage = 0.05  # Objectif de profit de 5%
trailing_stop_percentage = 0.02  # Trailing stop-loss de 2%

profit_manager = ProfitManagement(profit_target_percentage, trailing_stop_percentage)
profit_manager.set_entry_price(100)  # Prix d'entrée de la position

# Simulation du mouvement des prix
price_history = [100, 110, 120, 115, 125, 130, 135, 130, 140, 145]

for price in price_history:
    if profit_manager.take_profit(price):
        print(f"Prendre profit au prix {price}")
        break
    else:
        trailing_stop_price = profit_manager.trailing_stop_loss(price)
        print(f"Prix actuel : {price}, Trailing stop-loss : {trailing_stop_price}")
